In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Test de vulnérabilité sur un modèle convolutif maxout
#Entrainement sur le dataset de train CIFAR10
#Evaluation sur le dataset de test CIFAR10
#Evaluation sur le dataset de train CIFAR10 avec modif FGSM
#Evaluation sur le dataset de test CIFAR10 avec modif FGSM
#Ces evaluations sont effectuées pour les optimizer SGD et Adam

In [19]:
#Optimizer SGD

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
EPOCHS = 70
EPSILON = 0.1


# maxout convolutionnel
class MaxoutConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, k=2, kernel_size=3, padding=1):
        super().__init__()
        self.k = k
        self.out_channels = out_channels
        self.conv = nn.Conv2d(in_channels, out_channels * k,
                              kernel_size=kernel_size, padding=padding)

    def forward(self, x):
        x = self.conv(x)
        b, c, h, w = x.shape
        x = x.view(b, self.out_channels, self.k, h, w)
        x, _ = x.max(dim=2)
        return x


class RealConvMaxoutNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = MaxoutConv2d(3, 48, k=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(0.2)

        self.conv2 = MaxoutConv2d(48, 96, k=2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout(0.3)

        self.conv3 = MaxoutConv2d(96, 192, k=2)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.drop3 = nn.Dropout(0.4)

        self.fc = nn.Linear(192 * 4 * 4, 10)

    def forward(self, x):
        x = self.drop1(self.pool1(self.conv1(x)))
        x = self.drop2(self.pool2(self.conv2(x)))
        x = self.drop3(self.pool3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        return self.fc(x)

# cifar-10
def load_data():
    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,)*3, (0.5,)*3)
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,)*3, (0.5,)*3)
    ])

    trainset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform_train
    )
    testset = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=True, transform=transform_test
    )

    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=2)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE,
                            shuffle=False, num_workers=2)
    return trainloader, testloader

# training
def train(model, trainloader, optimizer, criterion, epochs):
    model.train()
    print(f"Entraînement sur {DEVICE}")

    for epoch in range(epochs):
        correct, total, running_loss = 0, 0, 0.0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

        acc = 100. * correct / total
        print(f"Epoch [{epoch+1}/{epochs}] | Loss {running_loss/len(trainloader):.3f} | Acc {acc:.2f}%")


# FGSM
def fgsm_attack(x, epsilon, grad):
    return x + epsilon * grad.sign()

# évaluation
def mean_confidence_on_errors(model, dataloader, adversarial=False, epsilon=0.0):
    model.eval()
    confidences = []

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        if adversarial:
            inputs.requires_grad = True
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)

            model.zero_grad()
            loss.backward()
            grad = inputs.grad.data
            inputs = fgsm_attack(inputs, epsilon, grad)
            outputs = model(inputs)
        else:
            with torch.no_grad():
                outputs = model(inputs)

        probs = torch.softmax(outputs, dim=1)
        preds = probs.argmax(dim=1)

        mask = preds != labels
        if mask.any():
            confidences.extend(probs[mask, preds[mask]].detach().cpu().numpy())

    if len(confidences) == 0:
        return 0.0
    return float(np.mean(confidences))

# précsion
def accuracy(model, dataloader, adversarial=False, epsilon=0.0):
    model.eval()
    correct, total = 0, 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        if adversarial:
            inputs.requires_grad = True
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            model.zero_grad()
            loss.backward()
            inputs = fgsm_attack(inputs, epsilon, inputs.grad.data)
            outputs = model(inputs)
        else:
            with torch.no_grad():
                outputs = model(inputs)

        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return 100. * correct / total

# MAIN
if __name__ == "__main__":
    trainloader, testloader = load_data()

    model = RealConvMaxoutNet().to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01,
                          momentum=0.9, weight_decay=1e-4)

    train(model, trainloader, optimizer, criterion, EPOCHS)

    acc_test_clean = accuracy(model, testloader, adversarial=False)
    acc_test_adv   = accuracy(model, testloader, adversarial=True, epsilon=EPSILON)
    acc_train_adv  = accuracy(model, trainloader, adversarial=True, epsilon=EPSILON)

    conf_test_clean = mean_confidence_on_errors(model, testloader, adversarial=False)
    conf_test_adv   = mean_confidence_on_errors(model, testloader, adversarial=True, epsilon=EPSILON)
    conf_train_adv  = mean_confidence_on_errors(model, trainloader, adversarial=True, epsilon=EPSILON)

    print("\n--- Résultats Maxout CIFAR-10 ---")
    print(f"Accuracy Test Clean   : {acc_test_clean:.2f}%")
    print(f"Accuracy Test FGSM    : {acc_test_adv:.2f}%")
    print(f"Accuracy Train FGSM   : {acc_train_adv:.2f}%\n")

    print("--- Confiance moyenne sur les erreurs ---")
    print(f"Confiance erreurs Test Clean : {conf_test_clean*100:.2f}%")
    print(f"Confiance erreurs Test FGSM  : {conf_test_adv*100:.2f}%")
    print(f"Confiance erreurs Train FGSM : {conf_train_adv*100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Entraînement sur cuda
Epoch [1/70] | Loss 1.652 | Acc 40.23%
Epoch [2/70] | Loss 1.244 | Acc 55.69%
Epoch [3/70] | Loss 1.060 | Acc 62.57%
Epoch [4/70] | Loss 0.953 | Acc 66.70%
Epoch [5/70] | Loss 0.870 | Acc 69.74%
Epoch [6/70] | Loss 0.811 | Acc 71.76%
Epoch [7/70] | Loss 0.763 | Acc 73.17%
Epoch [8/70] | Loss 0.722 | Acc 74.91%
Epoch [9/70] | Loss 0.693 | Acc 75.85%
Epoch [10/70] | Loss 0.670 | Acc 76.81%
Epoch [11/70] | Loss 0.644 | Acc 77.79%
Epoch [12/70] | Loss 0.629 | Acc 78.14%
Epoch [13/70] | Loss 0.607 | Acc 79.00%
Epoch [14/70] | Loss 0.592 | Acc 79.35%
Epoch [15/70] | Loss 0.582 | Acc 79.69%
Epoch [16/70] | Loss 0.566 | Acc 80.24%
Epoch [17/70] | Loss 0.551 | Acc 80.82%
Epoch [18/70] | Loss 0.539 | Acc 81.39%
Epoch [19/70] | Loss 0.531 | Acc 81.50%
Epoch [20/70] | Loss 0.528 | Acc 81.53%
Epoch [21/70] | Loss 0.509 | Acc 82.25%
Epoch [22/70] | Loss 0.506 | Acc 82.20%
Epoch [23/70] | Loss 0.499 | Ac

In [9]:
#Optimizer Adam

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
EPOCHS = 70
EPSILON = 0.1

# maxout conv
class MaxoutConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, k=2, kernel_size=3, padding=1):
        super().__init__()
        self.k = k
        self.out_channels = out_channels
        self.conv = nn.Conv2d(in_channels, out_channels * k,
                              kernel_size=kernel_size, padding=padding)

    def forward(self, x):
        x = self.conv(x)
        b, c, h, w = x.shape
        x = x.view(b, self.out_channels, self.k, h, w)
        x, _ = x.max(dim=2)
        return x


class RealConvMaxoutNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = MaxoutConv2d(3, 48, k=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(0.2)

        self.conv2 = MaxoutConv2d(48, 96, k=2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout(0.3)

        self.conv3 = MaxoutConv2d(96, 192, k=2)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.drop3 = nn.Dropout(0.4)

        self.fc = nn.Linear(192 * 4 * 4, 10)

    def forward(self, x):
        x = self.drop1(self.pool1(self.conv1(x)))
        x = self.drop2(self.pool2(self.conv2(x)))
        x = self.drop3(self.pool3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        return self.fc(x)

# cifar-10
def load_data():
    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,)*3, (0.5,)*3)
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,)*3, (0.5,)*3)
    ])

    trainset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform_train
    )
    testset = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=True, transform=transform_test
    )

    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=2)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE,
                            shuffle=False, num_workers=2)
    return trainloader, testloader

# training
def train(model, trainloader, optimizer, criterion, epochs):
    model.train()
    print(f"Entraînement sur {DEVICE}")

    for epoch in range(epochs):
        correct, total, running_loss = 0, 0, 0.0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

        acc = 100. * correct / total
        print(f"Epoch [{epoch+1}/{epochs}] | Loss {running_loss/len(trainloader):.3f} | Acc {acc:.2f}%")

# FGSM
def fgsm_attack(x, epsilon, grad):
    return x + epsilon * grad.sign()

# évaluation
def mean_confidence_on_errors(model, dataloader, adversarial=False, epsilon=0.0):
    model.eval()
    confidences = []

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        if adversarial:
            inputs.requires_grad = True
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)

            model.zero_grad()
            loss.backward()
            grad = inputs.grad.data
            inputs = fgsm_attack(inputs, epsilon, grad)
            outputs = model(inputs)
        else:
            with torch.no_grad():
                outputs = model(inputs)

        probs = torch.softmax(outputs, dim=1)
        preds = probs.argmax(dim=1)

        mask = preds != labels
        if mask.any():
            confidences.extend(probs[mask, preds[mask]].detach().cpu().numpy())

    if len(confidences) == 0:
        return 0.0
    return float(np.mean(confidences))

# précision
def accuracy(model, dataloader, adversarial=False, epsilon=0.0):
    model.eval()
    correct, total = 0, 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        if adversarial:
            inputs.requires_grad = True
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            model.zero_grad()
            loss.backward()
            inputs = fgsm_attack(inputs, epsilon, inputs.grad.data)
            outputs = model(inputs)
        else:
            with torch.no_grad():
                outputs = model(inputs)

        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return 100. * correct / total

# MAIN
if __name__ == "__main__":
    trainloader, testloader = load_data()

    model = RealConvMaxoutNet().to(DEVICE)
    criterion = nn.CrossEntropyLoss()

    # 🔁 CHANGEMENT ICI : SGD → Adam
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

    train(model, trainloader, optimizer, criterion, EPOCHS)

    acc_test_clean = accuracy(model, testloader, adversarial=False)
    acc_test_adv   = accuracy(model, testloader, adversarial=True, epsilon=EPSILON)
    acc_train_adv  = accuracy(model, trainloader, adversarial=True, epsilon=EPSILON)

    conf_test_clean = mean_confidence_on_errors(model, testloader, adversarial=False)
    conf_test_adv   = mean_confidence_on_errors(model, testloader, adversarial=True, epsilon=EPSILON)
    conf_train_adv  = mean_confidence_on_errors(model, trainloader, adversarial=True, epsilon=EPSILON)

    print("\n--- Résultats Maxout CIFAR-10 ---")
    print(f"Accuracy Test Clean   : {acc_test_clean:.2f}%")
    print(f"Accuracy Test FGSM    : {acc_test_adv:.2f}%")
    print(f"Accuracy Train FGSM   : {acc_train_adv:.2f}%\n")

    print("--- Confiance moyenne sur les erreurs ---")
    print(f"Confiance erreurs Test Clean : {conf_test_clean*100:.2f}%")
    print(f"Confiance erreurs Test FGSM  : {conf_test_adv*100:.2f}%")
    print(f"Confiance erreurs Train FGSM : {conf_train_adv*100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Entraînement sur cuda
Epoch [1/70] | Loss 1.397 | Acc 49.99%
Epoch [2/70] | Loss 1.005 | Acc 64.86%
Epoch [3/70] | Loss 0.875 | Acc 69.43%
Epoch [4/70] | Loss 0.801 | Acc 71.85%
Epoch [5/70] | Loss 0.747 | Acc 74.19%
Epoch [6/70] | Loss 0.707 | Acc 75.56%
Epoch [7/70] | Loss 0.668 | Acc 76.71%
Epoch [8/70] | Loss 0.655 | Acc 77.20%
Epoch [9/70] | Loss 0.622 | Acc 78.48%
Epoch [10/70] | Loss 0.613 | Acc 78.79%
Epoch [11/70] | Loss 0.595 | Acc 79.38%
Epoch [12/70] | Loss 0.582 | Acc 79.70%
Epoch [13/70] | Loss 0.570 | Acc 80.11%
Epoch [14/70] | Loss 0.553 | Acc 80.74%
Epoch [15/70] | Loss 0.547 | Acc 81.14%
Epoch [16/70] | Loss 0.536 | Acc 81.25%
Epoch [17/70] | Loss 0.534 | Acc 81.33%
Epoch [18/70] | Loss 0.522 | Acc 81.90%
Epoch [19/70] | Loss 0.511 | Acc 82.50%
Epoch [20/70] | Loss 0.506 | Acc 82.15%
Epoch [21/70] | Loss 0.500 | Acc 82.63%
Epoch [22/70] | Loss 0.492 | Acc 82.93%
Epoch [23/70] | Loss 0.488 | Ac

In [ ]:
#Test sur Rubbish Examples
#Génération de 10 000 échantillons avec du bruit gaussien
#Evaluation sur le réseau convnet entrainé au dessus (optimizer SGD executé en dernier)

In [20]:
print("\n--- Rubbish Class Test (N(0, I_{3×32×32}), 1000 samples) ---")

num_samples = 1000

# Bruit gaussien CIFAR-10 : (N, 3, 32, 32)
rubbish_noise = torch.randn(num_samples, 3, 32, 32).to(DEVICE)

model.eval()
with torch.no_grad():
    outputs = model(rubbish_noise)          # logits
    probs = torch.softmax(outputs, dim=1)   # probabilités
    confs, preds = probs.max(dim=1)         # confiance max par échantillon

# Une prédiction est considérée "fausse" si la confiance est élevée (> 0.5)
errors = confs > 0.5
error_rate = errors.float().mean().item() * 100

# Confiance moyenne sur ces erreurs
avg_conf_errors = confs[errors].mean().item() * 100 if errors.any() else 0.0

print("----------------------------------------------------------------------------------")
print(f"Nombre d'échantillons           : {num_samples}")
print(f"Taux de prédictions > 0.5       : {error_rate:.2f}%")
print(f"Confiance moyenne (sur erreurs) : {avg_conf_errors:.2f}%")
print("----------------------------------------------------------------------------------")

# Diagnostics utiles (sanity check)
print("Stats des probabilités softmax (rubbish inputs):")
print(f"  Min  : {probs.min().item():.4f}")
print(f"  Mean : {probs.mean().item():.4f}")
print(f"  Max  : {probs.max().item():.4f}")

print("Fraction de probs > 0.5 par classe :")
print((probs > 0.5).float().mean(dim=0).cpu().numpy())


--- Rubbish Class Test (N(0, I_{3×32×32}), 1000 samples) ---
----------------------------------------------------------------------------------
Nombre d'échantillons           : 1000
Taux de prédictions > 0.5       : 99.70%
Confiance moyenne (sur erreurs) : 97.60%
----------------------------------------------------------------------------------
Stats des probabilités softmax (rubbish inputs):
  Min  : 0.0000
  Mean : 0.1000
  Max  : 1.0000
Fraction de probs > 0.5 par classe :
[0.        0.        0.028     0.002     0.        0.        0.9480001
 0.        0.        0.019    ]
